In [12]:
from transformers import AutoTokenizer
from datasets import load_dataset
from config import config
import numpy as np

In [8]:
dataset = load_dataset("multi_nli")

Using custom data configuration default
Reusing dataset multi_nli (/home/macab/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39)


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9832
    })
})

In [15]:
np.unique(dataset['validation_matched']['label'], return_counts=True)[1]

array([3479, 3123, 3213])

In [16]:
np.unique(dataset['validation_mismatched']['label'], return_counts=True)[1]

array([3463, 3129, 3240])

In [7]:
set(dataset['train']['heuristic'])

{'constituent', 'lexical_overlap', 'subsequence'}

In [8]:
imdb = load_dataset("imdb")

Downloading: 4.75kB [00:00, 2.08MB/s]                   
Downloading: 2.17kB [00:00, 1.55MB/s]                   
                                           Dataset imdb downloaded and prepared to /home/macab/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


In [13]:
imdb

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [10]:
set(imdb['train']['label'])

{0, 1}

In [13]:
sst2 = load_dataset("glue", "sst2")

NameError: name 'load_dataset' is not defined

In [14]:
sst2

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'label', 'sentence'],
        num_rows: 1821
    })
})

In [16]:
set(sst2['train']['label'])

{0, 1}

In [17]:
sst2 = sst2.rename_column("sentence", "text")

In [18]:
sst2

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'text'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'label', 'text'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'label', 'text'],
        num_rows: 1821
    })
})

In [20]:
import torch

In [45]:
softmax_outputs = torch.randn((2, 2)).softmax(dim=-1)

In [46]:
softmax_outputs

tensor([[0.1517, 0.8483],
        [0.1810, 0.8190]])

In [47]:
index = torch.tensor([1, 0])

In [48]:
index.shape

torch.Size([2])

In [53]:
torch.index_select(
    input=softmax_outputs,
    dim=-1,
    index=index
)

tensor([[0.8483, 0.1517],
        [0.8190, 0.1810]])

In [57]:
softmax_outputs.select(dim=0, index=0)

tensor([0.1517, 0.8483])

In [1]:
# softmax_outputs.inde

In [6]:
from dataset.loader import create_loaders
from transformers import AutoTokenizer
from datasets import load_dataset
from config import config
from datasets import load_dataset
from dataset.imdb_sst2 import imdb_sst2_datasets, imdb_sst2_loaders
from dataset.paraphrase import paraphrase_datasets, paraphrase_loaders

In [2]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [3]:
# dsets = imdb_sst2_datasets(config=config['tasks']["imdb_sst2_sa"])
# loaders = imdb_sst2_loaders(
#     dataset=dsets,
#     config=config['tasks']["imdb_sst2_sa"],
#     tokenizer=tokenizer
# )

In [4]:
dsets = paraphrase_datasets(config=config['tasks']["paraphrase"])

Reusing dataset paws (/home/macab/.cache/huggingface/datasets/paws/labeled_final/1.1.0/09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34)
Reusing dataset glue (/home/macab/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 16/16 [00:00<00:00, 47.10ba/s]


In [6]:
from dataset.paraphrase import paraphrase_datasets, paraphrase_loaders

In [7]:
loaders = paraphrase_loaders(
    dataset=dsets,
    config=config['tasks']["paraphrase"],
    tokenizer=tokenizer
)

100%|██████████| 15/15 [00:11<00:00,  1.30ba/s]


In [8]:
 for domain in loaders:

        for set in loaders[domain]:

            print(f'domain = {domain} | set = {set} | len = {len(loaders[domain][set])}')

domain = qqp | set = train | len = 10915
domain = qqp | set = test | len = 3538
domain = qqp | set = valid | len = 3538
domain = paws | set = train | len = 10915
domain = paws | set = test | len = 3538
domain = paws | set = valid | len = 3538


In [6]:
batch = next(iter(loaders[domain]['train']))

In [9]:
tokenizer.decode((batch['input_ids'][0]), skip_special_tokens=True)

'cut through the layers of soap - opera emotion and you find a scathing portrayal of a powerful entity strangling the life out of the people who want to believe in it the most'

In [10]:
 for domain in loaders:

        for set in loaders[domain]:

            print(f'domain = {domain} | set = {set} | len = {len(loaders[domain][set])}')

domain = imdb | set = train | len = 6250
domain = imdb | set = valid | len = 214
domain = imdb | set = test | len = 214
domain = sst2 | set = train | len = 6250
domain = sst2 | set = valid | len = 214
domain = sst2 | set = test | len = 214


In [11]:
from transformers import AutoModel

In [12]:
model = AutoModel.from_pretrained('albert-base-v2')

Downloading: 100%|██████████| 684/684 [00:00<00:00, 414kB/s]
Downloading: 100%|██████████| 47.4M/47.4M [00:05<00:00, 8.86MB/s]
Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# model

In [3]:
# modules = [model.embeddings, *model.albert_layer_groups[:6]]

### Validate Dataset

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
from config import config
from dataset.mnli import mnli_datasets, mnli_loaders
from dataset.imdb_sst2 import imdb_sst2_datasets, imdb_sst2_loaders

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [3]:
mnli = mnli_datasets(config=config['tasks']['mnli'])

Using custom data configuration default
Reusing dataset multi_nli (/home/macab/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39)


In [4]:
for domain in mnli:

        for set in mnli[domain]:

            print(f'domain = {domain} | set = {set} | len = {type(mnli[domain][set])}')


domain = mnli | set = train | len = <class 'datasets.arrow_dataset.Dataset'>
domain = mnli | set = valid | len = <class 'datasets.arrow_dataset.Dataset'>
domain = mnli | set = test | len = <class 'datasets.arrow_dataset.Dataset'>


In [5]:
mnli

{'mnli': {'train': Dataset({
      features: ['premise', 'hypothesis', 'genre', 'label'],
      num_rows: 30000
  }),
  'valid': Dataset({
      features: ['premise', 'hypothesis', 'genre', 'label'],
      num_rows: 9815
  }),
  'test': Dataset({
      features: ['premise', 'hypothesis', 'genre', 'label'],
      num_rows: 9815
  })}}

In [6]:
mnli_loaders = mnli_loaders(dataset=mnli, config=config['tasks']['mnli'], tokenizer=tokenizer)

100%|██████████| 10/10 [00:06<00:00,  1.54ba/s]


In [7]:
for domain in mnli:

        for set in mnli[domain]:

            print(f'domain = {domain} | set = {set} | len = {len(mnli[domain][set])}')


domain = mnli | set = train | len = 7500
domain = mnli | set = valid | len = 2454
domain = mnli | set = test | len = 2454


In [5]:
sst2 = imdb_sst2_datasets(config=config['tasks']['imdb_sst2_sa'])

Reusing dataset glue (/home/macab/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [7]:
for domain in sst2:

        for set in sst2[domain]:

            print(f'domain = {domain} | set = {set} | len = {type(sst2[domain][set])}')


domain = sst2 | set = train | len = <class 'datasets.arrow_dataset.Dataset'>
domain = sst2 | set = valid | len = <class 'datasets.arrow_dataset.Dataset'>
domain = sst2 | set = test | len = <class 'datasets.arrow_dataset.Dataset'>


In [9]:
mnli_loaders = imdb_sst2_loaders(dataset=sst2, config=config['tasks']['imdb_sst2_sa'], tokenizer=tokenizer)

100%|██████████| 1/1 [00:00<00:00,  2.29ba/s]


In [12]:
for domain in mnli_loaders:

        for set in mnli_loaders[domain]:

            print(f'domain = {domain} | set = {set} | len = {type(mnli_loaders[domain][set])}')


domain = sst2 | set = train | len = <class 'torch.utils.data.dataloader.DataLoader'>
domain = sst2 | set = valid | len = <class 'torch.utils.data.dataloader.DataLoader'>
domain = sst2 | set = test | len = <class 'torch.utils.data.dataloader.DataLoader'>


In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("./outputs/mnli/mnli.csv")

In [10]:
df.head()

,text,ground_truth,distilbert_pred,distilbert_prob,roberta_pred,roberta_prob,bert_pred,bert_prob
0,the new rights are nice enough everyone really...,1,1,0.891813,1,0.745590,1,0.871587
1,this site includes a list of all award winners...,2,2,0.953285,2,0.989634,2,0.981063
2,uh i don't know i i have mixed emotions about ...,0,1,0.313883,0,0.689912,1,0.167071
3,yeah i i think my favorite restaurant is alway...,2,1,0.270101,1,0.300233,1,0.406287
4,i don't know um do you do a lot of camping i k...,2,2,0.951305,2,0.992169,2,0.987972


In [12]:
data = load_dataset("glue", "sst2")

Reusing dataset glue (/home/macab/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [13]:
data

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [2]:
# data['validation']['label']

In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("hans")

Reusing dataset hans (/home/macab/.cache/huggingface/datasets/hans/plain_text/1.0.0/1bbcb735c482acd54f2e118074b59cfd2bf5f7a5a285d4d540d1e632216672ac)


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
        num_rows: 30000
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
        num_rows: 30000
    })
})

In [8]:
set(dataset['validation']['label'])

{0, 1}

In [9]:
import numpy as np

In [10]:
# np.unique(dataset['validation']['label'], return_counts=True)

(array([0, 1]), array([15000, 15000]))

In [1]:
from dataset.hans import 

Reusing dataset hans (/home/macab/.cache/huggingface/datasets/hans/plain_text/1.0.0/1bbcb735c482acd54f2e118074b59cfd2bf5f7a5a285d4d540d1e632216672ac)


In [3]:
dataset

{'hans': {'train': Dataset({
      features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
      num_rows: 30000
  }),
  'valid': Dataset({
      features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
      num_rows: 10000
  }),
  'test': Dataset({
      features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
      num_rows: 10000
  })}}

In [6]:
set(dataset['hans']['train']['heuristic'])

{'constituent', 'lexical_overlap', 'subsequence'}

In [10]:
# set(dataset['hans']['train']['template'])

In [1]:
from config import config
from transformers import AutoTokenizer
from dataset.hans import hans_datasets, hans_loaders

In [2]:
dataset = hans_datasets(config=None)

Reusing dataset hans (/home/macab/.cache/huggingface/datasets/hans/plain_text/1.0.0/1bbcb735c482acd54f2e118074b59cfd2bf5f7a5a285d4d540d1e632216672ac)


In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
loaders, tokenized = hans_loaders(dataset=dataset, config=config['tasks']['hans'], tokenizer=tokenizer)

In [6]:
batch = next(iter(loaders['hans']['train']))

In [11]:
batch['label'].shape

torch.Size([4, 1])

In [11]:
len(batch['attention_mask'])

KeyError: 'attention_mask'

In [ ]:
batch

{'heuristic': ['lexical_overlap', 'subsequence', 'constituent', 'constituent'],
 'template': ['temp35', 'temp48', 'temp52', 'temp58']}

In [14]:
dataset['hans']['train'][0]

{'binary_parse_hypothesis': '( ( The banker ) ( ( avoided ( the professors ) ) . ) )',
 'binary_parse_premise': '( ( ( The banker ) ( avoided ( the professors ) ) ) ( , ( or ( ( ( the athlete ) ran ) . ) ) ) )',
 'heuristic': 'constituent',
 'hypothesis': 'The banker avoided the professors .',
 'label': 1,
 'parse_hypothesis': '(ROOT (S (NP (DT The) (NN banker)) (VP (VBD avoided) (NP (DT the) (NN professors))) (. .)))',
 'parse_premise': '(ROOT (S (S (NP (DT The) (NN banker)) (VP (VBD avoided) (NP (DT the) (NN professors)))) (, ,) (CC or) (S (NP (DT the) (NN athlete)) (VP (VBD ran))) (. .)))',
 'premise': 'The banker avoided the professors , or the athlete ran .',
 'subcase': 'cn_disjunction',
 'template': 'temp53'}

In [14]:
from dataset.HANSDataset import HANSDataset

In [15]:
dataset

{'hans': {'train': Dataset({
      features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
      num_rows: 30000
  }),
  'valid': Dataset({
      features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
      num_rows: 10000
  }),
  'test': Dataset({
      features: ['premise', 'hypothesis', 'label', 'parse_premise', 'parse_hypothesis', 'binary_parse_premise', 'binary_parse_hypothesis', 'heuristic', 'subcase', 'template'],
      num_rows: 10000
  })}}

In [16]:
new_dataset = HANSDataset(
    dataset=dataset['hans']['train'],
    tokenizer=tokenizer,
    max_len=128,
)

In [18]:
# new_dataset[0]['iput']

In [20]:
a  = tokenizer.decode(new_dataset[0]['input_ids'].tolist(), skip_special_tokens=True, );

In [21]:
a

'the banker avoided the professors, or the athlete ran. the banker avoided the professors.'

In [22]:
batch['template']

['temp61', 'temp38', 'temp9', 'temp25']